In [ ]:
from autodistill_yolov8 import YOLOv8
import supervision as sv

## best.pt로 detect시도

In [ ]:
import os
image_dir = './test/images'
file_list=os.listdir(image_dir)

model=YOLOv8("runs/detect/train/weights/best.pt")
result = model.predict("./test/images/left1011_jpg.rf.80329927f937d116686bc3a6f913274a.jpg")[0]
detections = sv.Detections.from_yolov8(result)


print(detections)

In [ ]:
def predict_and_save(img):
    import supervision as sv
    import cv2
    import torch
    
    image = cv2.imread(img)
    print(img)
    classes = ["water tank", "lift", "worker"]
    
    with torch.no_grad():
        model=YOLOv8("runs/detect/train/weights/best.pt")
        result = model.predict(img)[0]
        detections = sv.Detections.from_yolov8(result)
        labels = [f"{classes[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _ in detections]
        
    box_annotator = sv.BoxAnnotator()
    annotated_frame = box_annotator.annotate(scene=image.copy(), detections=detections, labels=labels)
    sv.plot_image(annotated_frame)
    
    parts = img.split("_jpg.rf.")
    parts = parts[0].split("/")[-1]
    output_path="./YOLO_result/"+parts+".png"
    print(output_path)
    cv2.imwrite(output_path, annotated_frame)
    
    return

In [ ]:
import os
test_list=os.listdir('./test/images')

    
for i in range(len(test_list)):
    predict_and_save('./test/images/'+test_list[i])

## Map 구하기 (test set)

In [ ]:
import supervision as sv
import numpy as np
from autodistill_yolov8 import YOLOv8

dataset = sv.DetectionDataset.from_yolo("./test/images/","./test/labels/","./data.yaml")

def callback(image: np.ndarray) -> sv.Detections:
    model=YOLOv8("runs/detect/train/weights/best.pt")
    result = model.predict(image)[0]
    return sv.Detections.from_yolov8(result)

from detection import MeanAveragePrecision
mean_average_precision = MeanAveragePrecision.benchmark(
dataset = dataset,
callback = callback
)


In [ ]:
print("map(50_95):", mean_average_precision.map50_95)
print("map(50):",mean_average_precision.map50)
print("map(75):",mean_average_precision.map75)
print("map(per_class):",mean_average_precision.per_class_ap50_95)